# Парсинг musik-produktiv.com

Данные для проекта будут запарсены с сайта musik-produktiv.com являющимся немецким интернет-магазином музыкальных инструментов. Для парсинга использована библиотека bs4 и ее модуль BeautifulSoup

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import copy

## Парсинг

Посмотрим на код первой страницы сайта.

In [2]:
response = requests.get('https://www.musik-produktiv.com/ru/bas-gitara/')
soup = BeautifulSoup(response.text)
#Выод на экран ограничен в этой и последующих ячейках ограничен по длине для читаемости ноутбука 
print(soup.prettify()[:1000])

<!DOCTYPE html>
<html lang="ru-RU">
 <head>
  <title>
   Бас-гитара | Musik Produktiv
  </title>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta charset="utf-8"/>
  <link as="style" href="https://sc1.musik-produktiv.com/css/mp-desktop-20220421-204657.min.css" rel="preload"/>
  <link href="https://sc1.musik-produktiv.com/css/mp-desktop-20220421-204657.min.css" rel="stylesheet"/>
  <link href="https://fonts.googleapis.com/css?family=Source+Sans+Pro:400,600&amp;subset=cyrillic&amp;display=block" rel="stylesheet"/>
  <style>
   .mp-sprite { background-image: url('https://sc1.musik-produktiv.com/img/css-sprites/online-shop-icons-20220226.png');}
  </style>
  <link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <script defer="" src="https://sc1.musik-produktiv.com/js/mp-desktop-20220421-164621.min.js">
  </script>
  <style>
   .mp-sprite-sp { background-image: url('https://sc1.musik-produktiv.com/img/css-sprites/online-shop-sp-2018-12-19-ru.pn

Для того, чтобы получить данные об инструментах не только с первой страницы раздела "бас-гитары", но и с остальных, необходимо получить список ссылок на каждую страницу сайта. Контект раздела занимает 10 страниц, но на первой странице оторбражены страницы 1,2,3,4,5...10. Найдем в коде страницы ссылки на эти страницы сайта и дополним их недостающими страницами 6,7,8,9.

In [3]:
response_pages = ['https://www.musik-produktiv.com/ru/bas-gitara/']
for i in range(len(soup.select('div.list_page a'))):
    response_pages.append(soup.select('div.list_page a')[i]['href'])
response_pages

['https://www.musik-produktiv.com/ru/bas-gitara/',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=2',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=3',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=4',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=5',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=11']

In [4]:
response_pages = response_pages[:5]
links = ['https://www.musik-produktiv.com/ru/bas-gitara/?p=6',
        'https://www.musik-produktiv.com/ru/bas-gitara/?p=7',
        'https://www.musik-produktiv.com/ru/bas-gitara/?p=8',
        'https://www.musik-produktiv.com/ru/bas-gitara/?p=9',
        'https://www.musik-produktiv.com/ru/bas-gitara/?p=10']
for link in links:
    response_pages.append(link)
response_pages

['https://www.musik-produktiv.com/ru/bas-gitara/',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=2',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=3',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=4',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=5',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=6',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=7',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=8',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=9',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=10']

Список страниц готов. Пройдемся по каждой странице и вытащим названия бас-гитар. Названия находятся в классe "main-right-4col" внутри ограничителей 'li', 'b'

In [5]:
bass_name_list = []
for link in response_pages:
    response = requests.get(link)
    soup = BeautifulSoup(response.text)
    for bass in range(len(soup.select('div.main-right-4col li'))):
        bass_name_list.append(soup.select('div.main-right-4col li b')[bass].text)
bass_name_list[:5]

['Höfner Ignition Club Bass VSB',
 'Höfner Violin Ignition Bass SB',
 'ESP LTD B-205 SM STBLKS',
 'Ibanez Gio GSR200B WNF',
 'Ibanez Artcore AGB200-NT']

Аналогично получим цены и ссылки на страницы продуктов. Из цен уберем пробелы и знаки "₽".

In [6]:
price_list = []
for link in response_pages:
    response = requests.get(link)
    soup = BeautifulSoup(response.text)
    for price in range(len(soup.select('div.main-right-4col li i'))):
        price_list.append(soup.select('div.main-right-4col li i')[price].text)
price_list[:5]

['₽ 25 388', '₽ 24 940', '₽ 50 432', '₽ 18 593', '₽ 41 742']

In [7]:
price_list_stripped = []
for price in price_list:
    price = price.replace('₽ ', '')
    price = price.replace(' ', '')
    price_list_stripped.append(price)
price_list = price_list_stripped
price_list[:5]

['25388', '24940', '50432', '18593', '41742']

In [8]:
bass_links_list = []
for link in response_pages:
    response = requests.get(link)
    soup = BeautifulSoup(response.text)
    for bass_link in range(len(soup.select('div.main-right-4col li'))):
        bass_links_list.append(soup.select('div.main-right-4col li a')[bass_link]['href'])
bass_links_list[:5]

['https://www.musik-produktiv.com/ru/hoefner-ignition-club-bass-vsb.html',
 'https://www.musik-produktiv.com/ru/hoefner-violin-ignition-bass-sb.html',
 'https://www.musik-produktiv.com/ru/esp-ltd-b-205-sm-stblks.html',
 'https://www.musik-produktiv.com/ru/ibanez-gio-gsr200b-wnf.html',
 'https://www.musik-produktiv.com/ru/ibanez-artcore-agb200-nt.html']

Теперь имея список ссылок на страницы проуктов зайдем на каждую из них и вытащим оттуда описания товаров. Посмотрим на случайную страницу товара и найдем в ней описание:

In [9]:
response = requests.get('https://www.musik-produktiv.com/ru/sandberg-california-tt4-mblk-rmn.html')
soup = BeautifulSoup(response.text)
str(soup)[18000:20000]

'"Sandberg"><span itemprop="name">Sandberg</span></a><meta content="3" itemprop="position"/></span><span itemprop="itemListElement" itemscope="" itemtype="http://schema.org/ListItem"><span itemprop="name">California TT4 MBLK RMN</span><meta content="https://www.musik-produktiv.com/ru/sandberg-california-tt4-mblk-rmn.html" itemprop="item"/><meta content="4" itemprop="position"/></span></div>\n<div>\n<div class="main-left-4col">\n<div class="product-image h-500">\n<img alt="Бас-гитара  Sandberg California TT4 MBLK RMN" height="480" src="https://sc1.musik-produktiv.com/img/clips/image-coming-soon-l.png" title="Бас-гитара  Sandberg California TT4 MBLK RMN" width="480"/></div>\n<div id="zoom-big"></div>\n<div class="product-info more-slider">\n<div class="more-slider-content h-500">\n<a class="markenlogo" href="https://www.musik-produktiv.com/ru/sandberg/" title="Sandberg"><img alt="Sandberg" height="50" src="https://sc1.musik-produktiv.com/img/logos/brands/sandberg.png" title="Sandberg" wi

Описание товара находится в классе "eigenschaften" внутри ограничителей "li". Пройдемся по каждому товару и вытащим опиания всех товаров.

In [10]:
description = []
for link in bass_links_list:
    response = requests.get(link)
    soup = BeautifulSoup(response.text)
    description.append(soup.select('ul.eigenschaften li'))
description[:3]

[[<li>Форма корпуса: Битлз бас</li>,
  <li>Материал корпуса: Клён (мэйпл)</li>,
  <li>Дека: Ель</li>,
  <li>Конструкция: Полуакустическая (Семи Холлоу Бади)</li>,
  <li>Цвет: Винтэйдж Санберст</li>,
  <li>Конструкция грифа: Закреплён винтами</li>,
  <li>Мензура: Короткая</li>,
  <li>Длина мензуры: 30,5" (777 мм)</li>,
  <li>Гриф: Трёхслойный клён/бук</li>,
  <li>Накладка грифа: Палисандр (розовое дерево)</li>,
  <li>Количество ладов: 22</li>,
  <li>Маркеры (вкладыши) грифа: Классические круглые</li>,
  <li>Число струн: четырёхструнная</li>,
  <li>Типы звукоснимателей: Хамбакер</li>,
  <li>Переключатель звукоснимателей: Переключатель типа слайд</li>,
  <li>Цвет фурнитуры: Никель</li>,
  <li>Страна производства: КНР</li>],
 [<li>Форма корпуса: Битлз бас</li>,
  <li>Материал корпуса: Клён (мэйпл)</li>,
  <li>Дека: Ель</li>,
  <li>Конструкция: Полуакустическая (Семи Холлоу Бади)</li>,
  <li>Цвет: Винтэйдж Санберст</li>,
  <li>Конструкция грифа: Вклеен в корпус</li>,
  <li>Мензура: Короткая

In [11]:
description_stripped = []
for bass in description:
    item_stripped = []
    for item in bass:
        item = str(item)
        item = item.replace('<li>', '')
        item = item.replace('</li>', '')
        item_stripped.append(item)
    description_stripped.append(item_stripped)
description = description_stripped
description[:3]

[['Форма корпуса: Битлз бас',
  'Материал корпуса: Клён (мэйпл)',
  'Дека: Ель',
  'Конструкция: Полуакустическая (Семи Холлоу Бади)',
  'Цвет: Винтэйдж Санберст',
  'Конструкция грифа: Закреплён винтами',
  'Мензура: Короткая',
  'Длина мензуры: 30,5" (777 мм)',
  'Гриф: Трёхслойный клён/бук',
  'Накладка грифа: Палисандр (розовое дерево)',
  'Количество ладов: 22',
  'Маркеры (вкладыши) грифа: Классические круглые',
  'Число струн: четырёхструнная',
  'Типы звукоснимателей: Хамбакер',
  'Переключатель звукоснимателей: Переключатель типа слайд',
  'Цвет фурнитуры: Никель',
  'Страна производства: КНР'],
 ['Форма корпуса: Битлз бас',
  'Материал корпуса: Клён (мэйпл)',
  'Дека: Ель',
  'Конструкция: Полуакустическая (Семи Холлоу Бади)',
  'Цвет: Винтэйдж Санберст',
  'Конструкция грифа: Вклеен в корпус',
  'Мензура: Короткая',
  'Длина мензуры: 30,5" (777 мм)',
  'Гриф: Трёхслойный клён/бук',
  'Накладка грифа: Палисандр (розовое дерево)',
  'Количество ладов: 22',
  'Маркеры (вкладыши

Убедимся, что признаки получены для всех товаров.

In [12]:
print(len(bass_name_list))
print(len(price_list))
print(len(bass_links_list))
print(len(description))

600
600
600
600


## Приведение данных к табличному виду

Чтобы получить таблицу, в которой каждый признак будет соответствовать нужному инструменту, приведем полученные данные в датафрейму, используя библиотку "Pandas".
Так как не каждое описание товара содержит все возможные признаки, необходимо провести подготовительую работу с описаниями товаров. Признаки и значения признака разделены двоеточием - разобьем их в разные колонки и посмотрим на то, насколько данные полные.

In [13]:
features = []
for bass in description:
    for item in bass:
        feature, sep, value = item.partition(':')
        features.append(feature)
features = pd.Series(features)

In [14]:
features.value_counts()

Число струн                                   598
Цвет                                          594
Конструкция грифа                             589
Конструкция                                   588
Мензура                                       587
Длина мензуры                                 586
Гриф                                          586
Количество ладов                              583
Накладка грифа                                581
Материал корпуса                              580
Цвет фурнитуры                                579
Форма корпуса                                 571
Маркеры (вкладыши) грифа                      544
Типы звукоснимателей                          535
Бриджевый звукосниматель                      519
Страна производства                           473
Звукосниматель грифа                          465
Встроенная электроника                        409
Элементы управления                           372
В комплекте поставки                          309


Не все признаки несут ценность в определении стоимости, уберем ненужные. Список всех возможным значений сохраним в "features".

In [15]:
features = list(features.unique())

unnecessary_values = ['Цвет',
             'Цвет фурнитуры',
             'Маркеры (вкладыши) грифа',
             'Элементы управления',
             'В комплекте поставки',
             'Переключатель звукоснимателей',
             'Отличительные особенности',
             'Толщина струн, установленных изготовителем',
             'Вес',
             'Серийный номер униката']

for value in unnecessary_values:
    features.remove(value)

features

['Форма корпуса',
 'Материал корпуса',
 'Дека',
 'Конструкция',
 'Конструкция грифа',
 'Мензура',
 'Длина мензуры',
 'Гриф',
 'Накладка грифа',
 'Количество ладов',
 'Число струн',
 'Типы звукоснимателей',
 'Страна производства',
 'Звукосниматель грифа',
 'Бриджевый звукосниматель',
 'Встроенная электроника',
 'Средний звукосниматель']

In [16]:
for i in range(len(features)):
    for bass in description:
        for item in bass:
            feature, sep, value = item.partition(':')
            if feature not in features:
                bass.remove(item)

Создадим списки пустых списков "features_bass" и "values_bass" равные количеству инструментов по длине.

In [17]:
features_bass = copy.deepcopy(description)
values_bass = copy.deepcopy(description)

In [18]:
for bass in range(len(features_bass)):
    features_bass[bass].clear()
    values_bass[bass].clear()

In [19]:
features_bass[:3]

[[], [], []]

In [20]:
values_bass[:3]

[[], [], []]

В "features_bass" добавим названия признаков каждой бас-гитары, а в "values_bass" добавим соответствующие значения признаков.

In [21]:
for bass in range(len(description)):
    for i in range(len(description[bass])):
        feature, sep, value = description[bass][i].partition(': ')
        features_bass[bass].append(feature)
        values_bass[bass].append(value)

In [22]:
features_bass[:3]

[['Форма корпуса',
  'Материал корпуса',
  'Дека',
  'Конструкция',
  'Конструкция грифа',
  'Мензура',
  'Длина мензуры',
  'Гриф',
  'Накладка грифа',
  'Количество ладов',
  'Число струн',
  'Типы звукоснимателей',
  'Страна производства'],
 ['Форма корпуса',
  'Материал корпуса',
  'Дека',
  'Конструкция',
  'Конструкция грифа',
  'Мензура',
  'Длина мензуры',
  'Гриф',
  'Накладка грифа',
  'Количество ладов',
  'Число струн',
  'Типы звукоснимателей',
  'Звукосниматель грифа',
  'Бриджевый звукосниматель',
  'Страна производства'],
 ['Форма корпуса',
  'Материал корпуса',
  'Дека',
  'Конструкция',
  'Конструкция грифа',
  'Мензура',
  'Длина мензуры',
  'Гриф',
  'Накладка грифа',
  'Количество ладов',
  'Число струн',
  'Типы звукоснимателей',
  'Звукосниматель грифа',
  'Бриджевый звукосниматель',
  'Встроенная электроника',
  'Страна производства']]

In [23]:
values_bass[:3]

[['Битлз бас',
  'Клён (мэйпл)',
  'Ель',
  'Полуакустическая (Семи Холлоу Бади)',
  'Закреплён винтами',
  'Короткая',
  '30,5" (777 мм)',
  'Трёхслойный клён/бук',
  'Палисандр (розовое дерево)',
  '22',
  'четырёхструнная',
  'Хамбакер',
  'КНР'],
 ['Битлз бас',
  'Клён (мэйпл)',
  'Ель',
  'Полуакустическая (Семи Холлоу Бади)',
  'Вклеен в корпус',
  'Короткая',
  '30,5" (777 мм)',
  'Трёхслойный клён/бук',
  'Палисандр (розовое дерево)',
  '22',
  'четырёхструнная',
  'Соупбар',
  'Хофнер Игнишн Стэйпл Никел',
  'Хофнер Игнишн Стэйпл Никел',
  'КНР'],
 ['Форма "Даблкатэвей"',
  'Ясень',
  'Клён, расщеплённый (неповторимый эффект)',
  'Солид Бади',
  'Закреплён винтами',
  'Длинная',
  '34" (864 мм)',
  'Пятислойный клён/орех',
  'Палисандр (розовое дерево)',
  '24',
  'пятиструнная',
  'Хамбакер',
  'ESP дизайнд SB-5N',
  'ESP дизайнд SB-5B',
  'Трёхполосный эквалайзер',
  'Индонезия']]

Представим список всех возможных признаков "features" в виде датафрейма "features_df"

In [24]:
features_df = pd.DataFrame({'features_bass' : features})
features_df

,features_bass
0,Форма корпуса
1,Материал корпуса
2,Дека
3,Конструкция
4,Конструкция грифа
5,Мензура
6,Длина мензуры
7,Гриф
8,Накладка грифа
9,Количество ладов


Для каждого из 600 инструментов создадим датафрейм, в котором в левом столбце окажутся все возможные признаки (соответствуют списку features_df), а в правом - все значения конктретного инструмента. Там, где принакуне будет соответствовать никакое значение - будет значение "Nan". И заменим значения в values_bass правым столбцом датафрейма.

In [25]:
for i in range(len(values_bass)):
    df = pd.DataFrame({'features_bass' : features_bass[i], 'values_bass' : values_bass[i]})
    df = pd.merge(features_df, df, how = 'outer')
    values_bass[i].clear()
    values_bass[i].append(list(df['values_bass']))

In [26]:
values_bass[:3]

[[['Битлз бас',
   'Клён (мэйпл)',
   'Ель',
   'Полуакустическая (Семи Холлоу Бади)',
   'Закреплён винтами',
   'Короткая',
   '30,5" (777 мм)',
   'Трёхслойный клён/бук',
   'Палисандр (розовое дерево)',
   '22',
   'четырёхструнная',
   'Хамбакер',
   'КНР',
   nan,
   nan,
   nan,
   nan]],
 [['Битлз бас',
   'Клён (мэйпл)',
   'Ель',
   'Полуакустическая (Семи Холлоу Бади)',
   'Вклеен в корпус',
   'Короткая',
   '30,5" (777 мм)',
   'Трёхслойный клён/бук',
   'Палисандр (розовое дерево)',
   '22',
   'четырёхструнная',
   'Соупбар',
   'КНР',
   'Хофнер Игнишн Стэйпл Никел',
   'Хофнер Игнишн Стэйпл Никел',
   nan,
   nan]],
 [['Форма "Даблкатэвей"',
   'Ясень',
   'Клён, расщеплённый (неповторимый эффект)',
   'Солид Бади',
   'Закреплён винтами',
   'Длинная',
   '34" (864 мм)',
   'Пятислойный клён/орех',
   'Палисандр (розовое дерево)',
   '24',
   'пятиструнная',
   'Хамбакер',
   'Индонезия',
   'ESP дизайнд SB-5N',
   'ESP дизайнд SB-5B',
   'Трёхполосный эквалайзер',
  

In [27]:
for i in range(len(values_bass)):
    values_bass[i] = values_bass[i][0]

In [28]:
values_bass[:3]

[['Битлз бас',
  'Клён (мэйпл)',
  'Ель',
  'Полуакустическая (Семи Холлоу Бади)',
  'Закреплён винтами',
  'Короткая',
  '30,5" (777 мм)',
  'Трёхслойный клён/бук',
  'Палисандр (розовое дерево)',
  '22',
  'четырёхструнная',
  'Хамбакер',
  'КНР',
  nan,
  nan,
  nan,
  nan],
 ['Битлз бас',
  'Клён (мэйпл)',
  'Ель',
  'Полуакустическая (Семи Холлоу Бади)',
  'Вклеен в корпус',
  'Короткая',
  '30,5" (777 мм)',
  'Трёхслойный клён/бук',
  'Палисандр (розовое дерево)',
  '22',
  'четырёхструнная',
  'Соупбар',
  'КНР',
  'Хофнер Игнишн Стэйпл Никел',
  'Хофнер Игнишн Стэйпл Никел',
  nan,
  nan],
 ['Форма "Даблкатэвей"',
  'Ясень',
  'Клён, расщеплённый (неповторимый эффект)',
  'Солид Бади',
  'Закреплён винтами',
  'Длинная',
  '34" (864 мм)',
  'Пятислойный клён/орех',
  'Палисандр (розовое дерево)',
  '24',
  'пятиструнная',
  'Хамбакер',
  'Индонезия',
  'ESP дизайнд SB-5N',
  'ESP дизайнд SB-5B',
  'Трёхполосный эквалайзер',
  nan]]

In [29]:
len(values_bass)

600

Проверим, что описание каждого инструмента включает в себя все 17 признаков.

In [30]:
summa = 0
for i in range(len(values_bass)):
    if len(values_bass[i]) != 17:
        summa += 1

In [31]:
summa

0

Создадим датафрейм "description_df", в котором строками будут инструменты, а столбцами - признаки.

In [32]:
description_df = pd.DataFrame(data = np.array(values_bass), columns = features)

In [33]:
description_df

,Форма корпуса,Материал корпуса,Дека,Конструкция,Конструкция грифа,Мензура,Длина мензуры,Гриф,Накладка грифа,Количество ладов,Число струн,Типы звукоснимателей,Страна производства,Звукосниматель грифа,Бриджевый звукосниматель,Встроенная электроника,Средний звукосниматель
0,Битлз бас,Клён (мэйпл),Ель,Полуакустическая (Семи Холлоу Бади),Закреплён винтами,Короткая,"30,5"" (777 мм)",Трёхслойный клён/бук,Палисандр (розовое дерево),22,четырёхструнная,Хамбакер,КНР,nan,nan,nan,nan
1,Битлз бас,Клён (мэйпл),Ель,Полуакустическая (Семи Холлоу Бади),Вклеен в корпус,Короткая,"30,5"" (777 мм)",Трёхслойный клён/бук,Палисандр (розовое дерево),22,четырёхструнная,Соупбар,КНР,Хофнер Игнишн Стэйпл Никел,Хофнер Игнишн Стэйпл Никел,nan,nan
2,"Форма ""Даблкатэвей""",Ясень,"Клён, расщеплённый (неповторимый эффект)",Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Пятислойный клён/орех,Палисандр (розовое дерево),24,пятиструнная,Хамбакер,Индонезия,ESP дизайнд SB-5N,ESP дизайнд SB-5B,Трёхполосный эквалайзер,nan
3,Саундджиэр,Нато (Ньято / Мора),nan,Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Клён (мэйпл),Ятоба,22,четырёхструнная,1 x P-Стайл + 1 x J-Стайл,Индонезия,PSNDP,PSNDJ,nan,nan
4,Полуакустическая (Семи Холлоу Бади),Клён (мэйпл),Клён (мэйпл),Полуакустическая (Семи Холлоу Бади),Вклеен в корпус,Короткая,"30,3"" (770 мм)",Трёхслойный клён/махагони,Индийский лорел (терминалия крылатая),22,четырёхструнная,Хамбакер,КНР,Ибанез клэссик элит,Ибанез клэссик элит,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,Спектор,Плакучий ясень,Тополь,Солид Бади,Сплошной с декой,Малтискейл,"Мультимензура ""Малтискэйл"" (веерные лады) 34"" ...",Пятислойный клён/венге,Венге,24,четырёхструнная,Хамбакер,Южная Корея,Фишмэн Флюэнс,Фишмэн Флюэнс,Двухполосный эквалайзер,nan
596,Спектор,Плакучий ясень,nan,Солид Бади,Закреплён винтами,Сверхдлинная,"35"" (889 мм)","3-слойный, выполненный из специально термическ...",Чёрное дерево (Эбони),24,пятиструнная,1 x P-Стайл + 1 x J-Стайл,Южная Корея,nan,EMG J,Двухполосный эквалайзер,EMG P
597,Спектор,Нато (Ньято / Мора),nan,Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Трёхслойный клён,Амарант,24,четырёхструнная,1 x P-Стайл + 1 x J-Стайл,КНР,nan,Спектор Джи,nan,Спектор P
598,Спектор,Нато (Ньято / Мора),nan,Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Трёхслойный клён,Амарант,24,четырёхструнная,1 x P-Стайл + 1 x J-Стайл,КНР,nan,Спектор Джи,nan,Спектор P


Присоединим полученный датафрейм к названиям инструментов и ценам.

In [34]:
musik_productiv_df = pd.DataFrame({'Наименование' : bass_name_list, 
                                  'Цена' : price_list_stripped})

In [35]:
musik_productiv_df = pd.merge(musik_productiv_df, description_df, how = 'left', left_index=True, right_index = True)

In [36]:
musik_productiv_df

,Наименование,Цена,Форма корпуса,Материал корпуса,Дека,Конструкция,Конструкция грифа,Мензура,Длина мензуры,Гриф,Накладка грифа,Количество ладов,Число струн,Типы звукоснимателей,Страна производства,Звукосниматель грифа,Бриджевый звукосниматель,Встроенная электроника,Средний звукосниматель
0,Höfner Ignition Club Bass VSB,25388,Битлз бас,Клён (мэйпл),Ель,Полуакустическая (Семи Холлоу Бади),Закреплён винтами,Короткая,"30,5"" (777 мм)",Трёхслойный клён/бук,Палисандр (розовое дерево),22,четырёхструнная,Хамбакер,КНР,nan,nan,nan,nan
1,Höfner Violin Ignition Bass SB,24940,Битлз бас,Клён (мэйпл),Ель,Полуакустическая (Семи Холлоу Бади),Вклеен в корпус,Короткая,"30,5"" (777 мм)",Трёхслойный клён/бук,Палисандр (розовое дерево),22,четырёхструнная,Соупбар,КНР,Хофнер Игнишн Стэйпл Никел,Хофнер Игнишн Стэйпл Никел,nan,nan
2,ESP LTD B-205 SM STBLKS,50432,"Форма ""Даблкатэвей""",Ясень,"Клён, расщеплённый (неповторимый эффект)",Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Пятислойный клён/орех,Палисандр (розовое дерево),24,пятиструнная,Хамбакер,Индонезия,ESP дизайнд SB-5N,ESP дизайнд SB-5B,Трёхполосный эквалайзер,nan
3,Ibanez Gio GSR200B WNF,18593,Саундджиэр,Нато (Ньято / Мора),nan,Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Клён (мэйпл),Ятоба,22,четырёхструнная,1 x P-Стайл + 1 x J-Стайл,Индонезия,PSNDP,PSNDJ,nan,nan
4,Ibanez Artcore AGB200-NT,41742,Полуакустическая (Семи Холлоу Бади),Клён (мэйпл),Клён (мэйпл),Полуакустическая (Семи Холлоу Бади),Вклеен в корпус,Короткая,"30,3"" (770 мм)",Трёхслойный клён/махагони,Индийский лорел (терминалия крылатая),22,четырёхструнная,Хамбакер,КНР,Ибанез клэссик элит,Ибанез клэссик элит,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,Spector NS Dimension MS 4 Super Faded Black,149267,Спектор,Плакучий ясень,Тополь,Солид Бади,Сплошной с декой,Малтискейл,"Мультимензура ""Малтискэйл"" (веерные лады) 34"" ...",Пятислойный клён/венге,Венге,24,четырёхструнная,Хамбакер,Южная Корея,Фишмэн Флюэнс,Фишмэн Флюэнс,Двухполосный эквалайзер,nan
596,Spector NS Pulse 5 Carbon Series Cinder Red,88037,Спектор,Плакучий ясень,nan,Солид Бади,Закреплён винтами,Сверхдлинная,"35"" (889 мм)","3-слойный, выполненный из специально термическ...",Чёрное дерево (Эбони),24,пятиструнная,1 x P-Стайл + 1 x J-Стайл,Южная Корея,nan,EMG J,Двухполосный эквалайзер,EMG P
597,Spector Performer 4 BLK,30541,Спектор,Нато (Ньято / Мора),nan,Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Трёхслойный клён,Амарант,24,четырёхструнная,1 x P-Стайл + 1 x J-Стайл,КНР,nan,Спектор Джи,nan,Спектор P
598,Spector Performer 4 MBL,32781,Спектор,Нато (Ньято / Мора),nan,Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Трёхслойный клён,Амарант,24,четырёхструнная,1 x P-Стайл + 1 x J-Стайл,КНР,nan,Спектор Джи,nan,Спектор P


In [37]:
musik_productiv_df.to_csv('musik_productiv_df', index = False)

## Вывод

Данные с сайта "musik-produktiv.com" получены и представляют собой единый датафрейм. Количество инструментов: 600, количество признаков: 19